# Autodock4 Fe: for metaloproteins docking

<p style="font-size:0.8em;line-height:50%">Gustavo E. Olivos Ramirez.</p>
<p style="font-size:0.8em;line-height:50%">Universidad Peruana Cayetano Heredia, Lima, Perú.</p>
<p style="font-size:0.8em;line-height:50%">gustavo.olivos@upch.pe</p>

<p style="text-align: justify;font-family: Comic;color:black;font-size:1.3em">This is a step-by-step tutorial to analyse protein interaction with metalic ion and ligand through molecular docking. The command lines can be runned from terminal in ubuntu. </p>

<td class="first">
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.1em; font-weight: bold">Software requirements:</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- Autodock version 4.2.6 or higher (sudo apt-get install autodock).</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- Autogrid version 4.2.6 or higher (sudo apt-get install autodogrid).</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- MGLTools version 1.5.7 or higher.</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.1em; font-weight: bold">Required input files: </p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- receptor.pdb. </p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- ligand.sdf. </p>
</td>
<td class="second">
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.1em; font-weight: bold">Scripts </p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- prepare_gpf4.py</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- prepare_dpf4.py</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- prepare_receptor4.py (optional)</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- pythonsh</p>
<p style="text-align: justify;font-family: Comic;color:black;font-size:1.0em">- AD4_parameters.dat</p>
</td>

## Introduction

<p style="text-align: justify;font-family: Comic;color:black;font-size:1.3em">Tuberculosis (TB), caused by<i> Mycobaterium (M.) tuberculosis</i>, represents one of the main causes of death in the world, and for which there exits a great interest for the development of strategies to reduce its prevalence. Pirazinamida represents one of the most important drug and is frecuently used for the treatment of this disease, since its use can reduce the time of treatment (from 9 to 6 moths) and it has been suggested as the unique drug that has an effect under latent bacteria state. In this sence, insights into its action mode at a molecular level need to be achieved. In this tutorial, a molecular docking using pyrazinamide (ligand) and pyrazinamidase (receptor) (PDB ID: 3PL1) is exemplified, to provide insights into atomistic interactions for futher analysis. </p> 

![alt text](receptor.png "Title")
<p style="text-align: center;font-family: Arial;color:black;font-size:1.2em;">Figure 1. Pirazinamide (sticks) and PZasa (cartoon) interaction, in presence of Fe<sup>+2</sup> (sphere).</p>

## Step one (1): Ligand preparation.

<p style="text-align: justify;font-family: Comic;color:black;font-size:1.3em">Pirazinamide is used as ligand and is retrieved from PubChem database (<a href="https://pubchem.ncbi.nlm.nih.gov/">https://pubchem.ncbi.nlm.nih.gov/</a>) (PubChem ID: 1046) in *.SDF format. To accomplish this task, PubChemPy tool is used. Moreover, the *.SDF file need to be transformed into *.PDBQT, and energetically minimized.</p> 

In [ ]:
%%bash
pip install PubChemPy
# This part is used to install PubChemPy.

In [ ]:
>>> from pubchempy import get_compounds, Compound
>>> comp = Compound.from_cid(1046)
>>> print(comp.isomeric_smiles)
# These commands show the smile form of the compound (not necessary to run).

In [ ]:
from pubchempy import *
download('SDF', 'ligand.sdf', '1046', 'cid')
# These commands download the ligand (pirazinamide) as ligand.sdf (2D format).

In [ ]:
%%bash
pymol='/home/schrodinger/Installers/pymol/bin/pymol'
$pymol ligand.sdf
# Now we can check the ligand structure with pymol 
# We assume that pymol is already installed and not explain this procedure.

In [ ]:
%%bash
babel -isdf ligand.sdf -opdb ligand.pdb --gen3D -p 7.4
# In this part, the lingand.sdf file is converted to ligand.pdb. 3D coordenates are generated at pH=7.4
# Babel is also assumed to be installed.
# A message like this is expected to appear: 1 molecule converted 125 audit log messages

In [ ]:
%%bash
pymol='/home/schrodinger/Installers/pymol/bin/pymol'
$pymol ligand.pdb
# Now we check the new generated structure with pymol.

In [ ]:
%%bash
./pythonsh prepare_ligand4.py -l ligand.pdb -o ligand.pdbqt -A hydrogens

In [ ]:
%%bash
obminimize -ff MMFF94 -n 5000 -sd -c 1e-9 ligand.pdbqt
# In this part we the ligand (pirazinamide) is energeticaly minimized.

In [ ]:
%%bash
pymol='/home/schrodinger/Installers/pymol/bin/pymol'
$pymol ligand.pdbqt
# Finally, we check the last structure generated, which will be used in the molecular docking procedure.

## Step two (2): Receptor preparation.

<p style="text-align: justify;font-family: Comic;color:black;font-size:1.3em">The receptor used in this part (PZase) is download from the Protein Data Bank (<a href="https://www.rcsb.org/">https://www.rcsb.org/</a>) (PDB ID: 3PL1) in *.PDB format. Besides, the receptor need to be prepared i</p> 

In [ ]:
%%bash
receptor=3PL1
wget https://files.rcsb.org/download/$receptor.pdb
# The wget command allow it to download the receptor remotely.

In [ ]:
%%bash
./pythonsh prepare_receptor4.py -r 3PL1.pdb -o 3PL1.pdbqt -A hydrogens

In [ ]:
%%bash
cat 3PL1.pdbqt | sed 's/0.000 Fe/2.000 Fe/' > tmp
mv tmp receptor.pdbqt
# In this part, we add charges to Fe ion.

In [ ]:
%%bash
adt="/home/schrodinger/Installers/mgltools_x86_64Linux2_1.5.7rc1/bin/./adt"
$adt receptor.pdbqt
# Now, in this step wi will define boxsize and grid-center manually.
# Take note of these values to use them in the following step.

In [ ]:
%%bash
pymol='/home/schrodinger/Installers/pymol/bin/pymol'
$pymol receptor.pdbqt
# Finally, we check the last receptor structure in PDBQT.

## Step three (3): Generating grid parameter file (.gpf).

In [ ]:
%%bash
./pythonsh prepare_gpf4.py -l ligand.pdbqt -r receptor.pdbqt -o grid_map.gpf -p npts=40,40,40 -p gridcenter=9.320,-26.529,-1.834 -p parameter_file=AD4_parameters.dat
# In this step, we define the grid parameters, considering the boxsize and gri-center.
# Parameters for iron ion also is considered.

## Step four (4): Running Autogrid.

In [ ]:
%%bash
autogrid4 -p grid_map.gpf -l grid_map.glg
# In this step, we generate the grid files.

## Step five (5): Generating docking parameter files (.dpf).

In [ ]:
%%bash
./pythonsh prepare_dpf4.py -l ligand.pdbqt -r receptor.pdbqt -p ga_run=100 -p ga_pop_size=350 -o genetics_algorithm.dpf
# In the next step, docking parameters, for genetics algorithm, is set up.

## Step six (6): Running Autodock.

In [ ]:
%%bash
autodock4 -p genetics_algorithm.dpf -l genetics_algorithm.dlg
# In this step, we run the genetics algorithm.

In [ ]:
%%bash
gedit genetics_algorithm.dlg
# Finally, we review the results.